# Connecting to the Congressional District Geospatial Data

In [6]:
# import filepath of congressional district geospatial data from config.py
from texas_gerrymandering_hb4.config import PLANC2308_SHP_FILE

# import geopandas to simplify working with geospatial data in the district shapefile
import geopandas as gpd

# load PLANC2308 Congressional district shapefile
gdf = gpd.read_file(PLANC2308_SHP_FILE)

# Converting the Congressional District Shapefile Into a Geopackage File

In [8]:
# import filepath of processed data directory from config.py
from texas_gerrymandering_hb4.config import PROCESSED_DATA_DIR

# save shapefile as a geopackage
gdf.to_file(PROCESSED_DATA_DIR/"districts.gpkg", driver="GPKG")